In [5]:

from project.data_extraction.dummy_data_extractor import extract_dummy_data
master_table = extract_dummy_data(r"C:\Users\ccarandang\OneDrive - Enova Facilities Management\Documents\GitHub\HubgradeDataCleaning\HubgradeDataCleaning\project\dummy_data")
master_table.at[0,"his"]

,ts,DMC Building 1 Data Quality Tests Dup of AHU_04_B1 Return Air Temp,DMC Building 1 Data Quality Tests New-Point
0,2023-02-24 18:45:00+04:00,24.420517,NaN
1,2023-02-24 18:50:00+04:00,24.420517,NaN
2,2023-02-24 18:55:00+04:00,24.420517,NaN
3,2023-02-24 19:00:00+04:00,24.345844,NaN
4,2023-02-24 19:05:00+04:00,24.271170,NaN
...,...,...,...
4391,2023-03-12 00:40:00+04:00,24.119339,31.959213
4392,2023-03-12 00:45:00+04:00,24.119339,33.179718
4393,2023-03-12 00:50:00+04:00,24.119339,35.324806
4394,2023-03-12 00:55:00+04:00,24.119339,33.506237
